### Data is coming from around 400 stocks of miniute-level information from 2023/4/1 to 2025/4/1.

### Steps to create docker
### We used docker desktop to gain data from polygon.io
### Docker container info
### condescending_carver
### container_id : 526d448ad4377e88be3ad614560dc4fa0a3ea42cef5bdf50da123f167bd0d4d6
### 1. We pick the tickers, time period and frequency we need for the data
### 2. We write a for loop to gain each ticker info orderly and save it as ".py"
### 3. We write a Dockerfile which 
###    FROM python:3.11-slim
###    WORKDIR /app
###    COPY fetch_stocks.py /app/
###    RUN pip install pandas requests
###    VOLUME ["/data"]
####   ENV POLYGON_API_KEY=""
###    CMD ["python", "fetch_stocks.py"]

# Part 1. Data Clean

### Check the exact numbers of csv files we have 

In [1]:
import os
import pandas as pd

files = os.listdir(r'C:\Users\Username\OneDrive\Desktop\810project')
print(f"Total csv files: {len([f for f in files if f.endswith('.csv')])}")

Total csv files: 428


### Merge all csv to one for the convenience of following processes

In [ ]:
import pandas as pd

path = r'C:\Users\Username\OneDrive\Desktop\810project'
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]

all_data = []
for file in csv_files:
    df = pd.read_csv(os.path.join(path, file))
    ticker = file.replace(".csv", "")
    df["ticker"] = ticker
    all_data.append(df)

merged_df = pd.concat(all_data, ignore_index=True)
merged_df.to_csv(r'C:\Users\Username\OneDrive\Desktop\810project\merged_raw.csv', index=False)

### Too slow, transfrom to Parquet to improve efficiency

In [ ]:
merged_df.to_parquet(r'C:\Users\Username\OneDrive\Desktop\810project\merged_raw.parquet', index=False)

In [1]:
import pandas as pd
df_par = pd.read_parquet(r'C:\Users\Username\OneDrive\Desktop\810project\mermiss_raw.parquet')

### Simply view the data

In [2]:
df_par.shape

(168836922, 10)

In [3]:
df_par.head(10)

,v,vw,o,c,h,l,t,n,datetime,ticker
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A
5,2483.0,136.7879,137.215,136.6100,137.215,136.61,1680528900000,74,2023-04-03 13:35:00,A
6,4375.0,136.4643,136.530,136.7150,136.715,136.34,1680528960000,72,2023-04-03 13:36:00,A
7,4552.0,136.6168,136.670,136.4700,136.870,136.28,1680529020000,80,2023-04-03 13:37:00,A
8,1157.0,136.4493,136.320,136.4450,136.465,136.32,1680529080000,13,2023-04-03 13:38:00,A
9,4688.0,136.6723,136.580,136.9800,136.980,136.55,1680529140000,71,2023-04-03 13:39:00,A


In [4]:
df_par.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168836922 entries, 0 to 168836921
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   v         float64
 1   vw        float64
 2   o         float64
 3   c         float64
 4   h         float64
 5   l         float64
 6   t         int64  
 7   n         int64  
 8   datetime  object 
 9   ticker    object 
dtypes: float64(6), int64(2), object(2)
memory usage: 12.6+ GB


In [6]:
df_par.isna().sum()

v           209
vw          190
o           218
c           198
h           216
l           219
t             0
n             0
datetime      0
ticker        0
dtype: int64

### Rename the columns name for better viewing

In [7]:
df_par = df_par.rename(columns={
    'v': 'volume',
    'vw': 'vwap',
    'o': 'open',
    'c': 'close',
    'h': 'high',
    'l': 'low',
    't': 'timestamp',
    'n': 'transactions'
})
df_par.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A


### We are starting to deal with the missing values

### Since the data is min-level stock info, for the price columns, we use ffill to deal with
### Starting as a quant, we need to ensure the truth of data, so just apply ffill

In [8]:
# define the price cols 
price_cols = ['open', 'close', 'high', 'low']

# use for loop to ffill
for col in price_cols:
    df_par[col] = df_par.groupby('ticker')[col].ffill()

# check again the data
df_par.isna().sum()

volume          209
vwap            190
open              0
close             0
high              0
low               0
timestamp         0
transactions      0
datetime          0
ticker            0
dtype: int64

### For the volume, just fill 0 since it does not matter
### For the volume weighted average price, we apply linear interpoate here

In [9]:
df_par['volume'] = df_par['volume'].fillna(0)

# the vwap need to interpolate in the same ticker
df_par['vwap'] = df_par.groupby('ticker')['vwap'].transform(lambda group: group.interpolate(method='linear'))

df_par.isna().sum()

volume          0
vwap            0
open            0
close           0
high            0
low             0
timestamp       0
transactions    0
datetime        0
ticker          0
dtype: int64

### Now the data seems good!

### Save the clean df so that we do not need to clean again!

In [ ]:
df_par.to_parquet(r'C:\Users\Username\OneDrive\Desktop\810project\merged_cleaned.parquet', index=False)

### Try to use Spark to do data clean

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("810Project") \
    .master("local[*]") \
    .config("spark.driver.memory", "24g") \
    .getOrCreate()

In [11]:
df_spark = spark.read.parquet(r'C:\Users\Username\OneDrive\Desktop\810project\mermiss_raw.parquet')

df_spark.show(10)

+-------+--------+-------+--------+-------+------+-------------+---+-------------------+------+
|      v|      vw|      o|       c|      h|     l|            t|  n|           datetime|ticker|
+-------+--------+-------+--------+-------+------+-------------+---+-------------------+------+
|17632.0|137.0174| 137.42|  137.26|137.565|136.85|1680528600000|105|2023-04-03 13:30:00|     A|
| 1011.0| 137.253|137.255|  137.32| 137.32|137.25|1680528660000| 28|2023-04-03 13:31:00|     A|
|  697.0|137.2364| 137.22|  137.33| 137.33|136.95|1680528720000| 18|2023-04-03 13:32:00|     A|
| 4004.0|137.3055| 136.99|  137.57| 137.57|136.99|1680528780000| 79|2023-04-03 13:33:00|     A|
| 7244.0|137.4399| 137.69|137.3784| 137.72|137.11|1680528840000|107|2023-04-03 13:34:00|     A|
| 2483.0|136.7879|137.215|  136.61|137.215|136.61|1680528900000| 74|2023-04-03 13:35:00|     A|
| 4375.0|136.4643| 136.53| 136.715|136.715|136.34|1680528960000| 72|2023-04-03 13:36:00|     A|
| 4552.0|136.6168| 136.67|  136.47| 136.

In [12]:
rename_dict = {
    "v": "volume",
    "vw": "vwap",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "timestamp",
    "n": "transactions"
}
for old_name, new_name in rename_dict.items():
    df_spark = df_spark.withColumnRenamed(old_name, new_name)
df_spark.show(5)

+-------+--------+-------+--------+-------+------+-------------+------------+-------------------+------+
| volume|    vwap|   open|   close|   high|   low|    timestamp|transactions|           datetime|ticker|
+-------+--------+-------+--------+-------+------+-------------+------------+-------------------+------+
|17632.0|137.0174| 137.42|  137.26|137.565|136.85|1680528600000|         105|2023-04-03 13:30:00|     A|
| 1011.0| 137.253|137.255|  137.32| 137.32|137.25|1680528660000|          28|2023-04-03 13:31:00|     A|
|  697.0|137.2364| 137.22|  137.33| 137.33|136.95|1680528720000|          18|2023-04-03 13:32:00|     A|
| 4004.0|137.3055| 136.99|  137.57| 137.57|136.99|1680528780000|          79|2023-04-03 13:33:00|     A|
| 7244.0|137.4399| 137.69|137.3784| 137.72|137.11|1680528840000|         107|2023-04-03 13:34:00|     A|
+-------+--------+-------+--------+-------+------+-------------+------------+-------------------+------+
only showing top 5 rows



In [14]:
df_spark.createOrReplaceTempView("market")

# Use SQL to check the na value
spark.sql("""
SELECT
    SUM(CASE WHEN open IS NULL THEN 1 ELSE 0 END) AS null_open,
    SUM(CASE WHEN high IS NULL THEN 1 ELSE 0 END) AS null_high,
    SUM(CASE WHEN low IS NULL THEN 1 ELSE 0 END) AS null_low,
    SUM(CASE WHEN close IS NULL THEN 1 ELSE 0 END) AS null_close,
    SUM(CASE WHEN volume IS NULL THEN 1 ELSE 0 END) AS null_volume,
    SUM(CASE WHEN vwap IS NULL THEN 1 ELSE 0 END) AS null_vwap
FROM market
""").show()

# Check if any of the stock did not make any order
spark.sql("""
SELECT ticker, COUNT(*) AS cnt, SUM(volume) AS total_volume
FROM market
GROUP BY ticker
HAVING total_volume = 0
""").show()

# Use SQL to check how many tickers there are
spark.sql("""
SELECT COUNT(DISTINCT ticker) AS unique_ticker_count FROM market
""").show()

+---------+---------+--------+----------+-----------+---------+
|null_open|null_high|null_low|null_close|null_volume|null_vwap|
+---------+---------+--------+----------+-----------+---------+
|      218|      216|     219|       198|        209|      190|
+---------+---------+--------+----------+-----------+---------+

+------+---+------------+
|ticker|cnt|total_volume|
+------+---+------------+
+------+---+------------+

+-------------------+
|unique_ticker_count|
+-------------------+
|                429|
+-------------------+



In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import last, col
import sys

w = Window.partitionBy("ticker").orderBy("datetime").rowsBetween(-sys.maxsize, 0)

# forward fill open/high/low/close
for colname in ["open", "high", "low", "close"]:
    df_spark = df_spark.withColumn(colname, last(col(colname), ignorenulls=True).over(w))


In [16]:
from pyspark.sql.functions import lag, lead, when

df_spark = df_spark.fillna({'volume': 0})

w = Window.partitionBy("ticker").orderBy("datetime")
# linear interpolation for vwap 
df_spark = df_spark.withColumn("vwap_prev", lag("vwap", 1).over(w))
df_spark = df_spark.withColumn("vwap_next", lead("vwap", 1).over(w))
df_spark = df_spark.withColumn(
    "vwap_interp",
    when(col("vwap").isNull(), (col("vwap_prev") + col("vwap_next")) / 2).otherwise(col("vwap"))
).drop("vwap").withColumnRenamed("vwap_interp", "vwap")

df_spark.show(5)

+-------+------+------+------+-----+-------------+------------+-------------------+------+---------+---------+-------+
| volume|  open| close|  high|  low|    timestamp|transactions|           datetime|ticker|vwap_prev|vwap_next|   vwap|
+-------+------+------+------+-----+-------------+------------+-------------------+------+---------+---------+-------+
|  284.0|  85.5|  85.5|  85.5| 85.5|1680519600000|          16|2023-04-03 11:00:00|   GIS|     NULL|  85.3907|85.5081|
|  713.0| 85.37| 85.37| 85.37|85.37|1680528540000|          17|2023-04-03 13:29:00|   GIS|  85.5081|  85.4951|85.3907|
|85741.0| 85.49|85.555| 85.65|85.43|1680528600000|         275|2023-04-03 13:30:00|   GIS|  85.3907|  85.5402|85.4951|
| 6811.0| 85.57| 85.47| 85.66|85.45|1680528660000|         127|2023-04-03 13:31:00|   GIS|  85.4951|   85.393|85.5402|
| 3919.0|85.475| 85.36|85.475|85.35|1680528720000|          95|2023-04-03 13:32:00|   GIS|  85.5402|  85.2405| 85.393|
+-------+------+------+------+-----+------------

In [17]:
spark.stop()

# Part 2: Feature Engineering

### Time Features

In [31]:
df_par["datetime"] = pd.to_datetime(df_par["datetime"])

df_par["minute"] = df_par["datetime"].dt.minute
df_par["hour"] = df_par["datetime"].dt.hour
df_par["dayofweek"] = df_par["datetime"].dt.dayofweek
df_par["is_open_hour"] = df_par["hour"].between(9, 16)

df_par.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker,minute,hour,dayofweek,is_open_hour
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A,30,13,0,True
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A,31,13,0,True
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A,32,13,0,True
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A,33,13,0,True
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A,34,13,0,True


### Price Features

In [32]:
import numpy as np
df_par["hl_spread"] = df_par["high"] - df_par["low"]
df_par["oc_return"] = (df_par["close"] - df_par["open"]) / df_par["open"]

df_par["log_return"] = (
    df_par.groupby("ticker")["close"]
    .apply(lambda x: np.log(x / x.shift(1)))
    .reset_index(drop=True)
)
df_par.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker,minute,hour,dayofweek,is_open_hour,hl_spread,oc_return,log_return
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A,30,13,0,True,0.715,-0.001164,NaN
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A,31,13,0,True,0.070,0.000474,0.000437
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A,32,13,0,True,0.380,0.000802,0.000073
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A,33,13,0,True,0.580,0.004234,0.001746
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A,34,13,0,True,0.610,-0.002263,-0.001394


### Lag Features

In [33]:
df_par["close_lag1"] = (
    df_par.groupby("ticker")["close"]
    .shift(1)
    .reset_index(drop=True)
)


df_par["close_roll_mean_5"] = (
    df_par.groupby("ticker")["close"]
    .transform(lambda x: x.rolling(5).mean())
    .reset_index(drop=True)
)


df_par["volume_roll_std_15"] = (
    df_par.groupby("ticker")["volume"]
    .transform(lambda x: x.rolling(15).std())
    .reset_index(drop=True)
)

df_par.head()

,volume,vwap,open,close,high,low,timestamp,transactions,datetime,ticker,minute,hour,dayofweek,is_open_hour,hl_spread,oc_return,log_return,close_lag1,close_roll_mean_5,volume_roll_std_15
0,17632.0,137.0174,137.420,137.2600,137.565,136.85,1680528600000,105,2023-04-03 13:30:00,A,30,13,0,True,0.715,-0.001164,NaN,NaN,NaN,NaN
1,1011.0,137.2530,137.255,137.3200,137.320,137.25,1680528660000,28,2023-04-03 13:31:00,A,31,13,0,True,0.070,0.000474,0.000437,137.26,NaN,NaN
2,697.0,137.2364,137.220,137.3300,137.330,136.95,1680528720000,18,2023-04-03 13:32:00,A,32,13,0,True,0.380,0.000802,0.000073,137.32,NaN,NaN
3,4004.0,137.3055,136.990,137.5700,137.570,136.99,1680528780000,79,2023-04-03 13:33:00,A,33,13,0,True,0.580,0.004234,0.001746,137.33,NaN,NaN
4,7244.0,137.4399,137.690,137.3784,137.720,137.11,1680528840000,107,2023-04-03 13:34:00,A,34,13,0,True,0.610,-0.002263,-0.001394,137.57,137.37168,NaN


### Store the results

In [ ]:
df_par.to_parquet(r'C:\Users\Username\OneDrive\Desktop\810project\dffeatures.parquet', index=False)